<a href="https://colab.research.google.com/github/mhpbreugem/BBP/blob/main/Admati_1985.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
using LinearAlgebra, BenchmarkTools, Symbolics, NLsolve

In [75]:
function φSYS(Ψ,ΣF,rf,γ)

    ## Notes

        ## TODO: Generalize: All RV's have zero mean.
        ## Z is random supply not demand

    # 00. Extract Variables and Define Random Variables

        a11,a12,a13,a14,a21,a22,a23,a24,b11,b12,b13,b14,b21,b22,b23,b24,c11,c12,c13,c14,c21,c22,c23,c24,d11,d12,d13,d14,d21,d22,d23,d24,ΣΠ11,ΣΠ12,ΣΠ21,ΣΠ22 = Ψ

        # F = [Π1, Π2, Z1, Z2, ε1i, ε2i]

    # 0. Define Private Information

        S_PRIVATE1 = [1,0,0,0,1,0]     # Signal Y1: F.S_PRIVATE1
        S_PRIVATE2 = [0,1,0,0,0,1]     # Signal Y1: F.S_PRIVATE1

    # 1. Conjecture Price Function (Public Information)

        ## P1 = a11*Π1 + a12*Π2 + a13*Z1 + a14*Z2
        ## P2 = a21*Π1 + a22*Π2 + a23*Z1 + a24*Z2

        S_PUBLIC1 = [a11, a12, a13, a14, 0, 0]          # Signal P1: F.S_PUBLIC
        S_PUBLIC2 = [a21, a22, a23, a24, 0, 0]          # Signal P1: F.S_PUBLIC

    # 2. Learning

        ## E[Π1|S] = b11*Y1 + b12*Y2 + b13*P1 + b14*P2
        ## E[Π2|S] = b21*Y1 + b22*Y2 + b23*P1 + b24*P2

        S = hcat(S_PRIVATE1, S_PRIVATE2, S_PUBLIC1, S_PUBLIC2)

        ΣYY = transpose(S) * ΣF * S                # 4x4
        ΣFY = ΣF * S                               # 6×4
        B = ΣFY * inv(ΣYY)                         # 6×4
        Σ = ΣF - ΣFY * inv(ΣYY) * transpose(ΣFY)   # 6×6

        EQ_LEARN = [b11 - B[1,1], b12 - B[1,2], b13 - B[1,3], b14 - B[1,4], b21 - B[2,1], b22 - B[2,2], b23 - B[2,3], b24 - B[2,4], ΣΠ11 - Σ[1,1], ΣΠ12 - Σ[1,2], ΣΠ21 - Σ[2,1], ΣΠ22 - Σ[2,2]]

    # 3. Individual Demand (CARA)

        b1 = [b11 b12;b21 b22]
        b2 = [b13 b14;b23 b24]
        ΣΠ = [ΣΠ11 ΣΠ12;ΣΠ21 ΣΠ22]

        ## Xi = {ΣΠ⁻¹ . [E[Π|S] - P*(1+rf)]} / γ
        ##    = ΣΠ⁻¹ . {b1*Y + [b2 - I*(1+rf)]*P} / γ
        ##    = c1*Y1 + c2*P

        C1 = inv(ΣΠ) * b1 / γ
        C2 = inv(ΣΠ) * (b2 - (1+rf)*I(2)) / γ

        EQ_DEMAND = [c11 - C1[1,1], c12 - C1[1,2],c21 - C1[2,1], c22 - C1[2,2], c13 - C2[1,1], c14 - C2[1,2],c23 - C2[2,1], c24 - C2[2,2]]

    # 4. Demand Aggregation

        ## Due to i.i.d. εi, μi aggregates to Π
        ## Z = d1*Π + d2*P

        EQ_AGGREGATION = [d11 - c11, d12 - c12, d13 - c13, d14 - c14, d21 - c21, d22 - c22, d23 - c23, d24 - c24]

    # 5. Market Clearing

        ## P = (-d1/d2)*Π + (1/d2)*Z
        ##   = a1*Π + a2*Z

        D1 = [d11 d12;d21 d22]
        D2 = [d13 d14;d23 d24]

        A1 = -inv(D2)*D1
        A2 = inv(D2)

        EQ_MC = [a11 - A1[1,1], a12 - A1[1,2],a21 - A1[2,1], a22 - A1[2,2], a13 - A2[1,1], a14 - A2[1,2],a23 - A2[2,1], a24 - A2[2,2]]

    return vcat(EQ_LEARN, EQ_DEMAND, EQ_AGGREGATION,EQ_MC)

end

φSYS (generic function with 1 method)

In [77]:
#φSYS(0.5*rand(36),diagm([1,1,1,1,1,1]),0,1)

In [65]:
x0_2stocks = [
    # a11 a12 a13 a14  (Price P1 as function of Π1, Π2, Z1, Z2)
     0.6667, 0, -0.6667, 0.0,
    # a21 a22 a23 a24  (Price P2 as function of Π1, Π2, Z1, Z2)
    0,  0.6667, 0, -0.6667,

    # b11 b12 b13 b14  (Π1 as function of Y1, Y2, P1, P2)
     0.5,  0.0, 0.0, 0.5,
    # b21 b22 b23 b24  (Π2 as function of Y1, Y2, P1, P2)
     -1.5,  0.0, 0.0, -1.5,

    # c11 c12 c13 c14  (X1 = c11*Y1 + c12*Y2 + c13*P1 + c14*P2)
     1.0,  0.0,  0.5,  0.0,
    # c21 c22 c23 c24  (X2 = ...)
     0.0,  1.0,  0.0,  0.5,

    # d11 d12 d13 d14  (Z1 = d11*Π1 + d12*Π2 + d13*P1 + d14*P2)
     1.0,  0.0,  0.5,  0.0,
    # d21 d22 d23 d24  (Z2 = ...)
     0.0,  1.0,  0.0,  0.5,

    # ΣΠ11 ΣΠ12 ΣΠ21 ΣΠ22 (Covariance of Π)
     0.3333, 0.0,
     0.0,    0.3333
]


36-element Vector{Float64}:
  0.6667
  0.0
 -0.6667
  0.0
  0.0
  0.6667
  0.0
 -0.6667
  0.5
  0.0
  0.0
  0.5
 -1.5
  ⋮
  1.0
  0.0
  0.5
  0.0
  0.0
  1.0
  0.0
  0.5
  0.3333
  0.0
  0.0
  0.3333

In [80]:
x0 = 0.5*rand(36)
sol = nlsolve(x -> φSYS(x, diagm([1,1,1,1,1,1]),0,1),x0_2stocks )

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.6667, 0.0, -0.6667, 0.0, 0.0, 0.6667, 0.0, -0.6667, 0.5, 0.0, 0.0, 0.5, -1.5, 0.0, 0.0, -1.5, 1.0, 0.0, 0.5, 0.0, 0.0, 1.0, 0.0, 0.5, 1.0, 0.0, 0.5, 0.0, 0.0, 1.0, 0.0, 0.5, 0.3333, 0.0, 0.0, 0.3333]
 * Zero: [-1845.2075280961112, 0.825716932073008, 2174.51987547597, -5.078419691405735, -380.3446719522932, -116.1634732109364, 448.7107191148763, 134.1306174522513, 0.47257897445266034, 0.0006787017769875838, 0.529300020393543, -0.00011943684230187363, -0.0009226342788937971, 0.474448660047952, -0.00044034469121712104, 0.5334322983628119, 0.7785642659706355, 0.0012639349821348467, -0.35265143668233334, -2.2017732226470957e-5, -0.00234171355809976, 0.7879269088008185, -0.0009757967539266928, -0.34798470504284956, 0.8485510269931732, 0.001630149203790004, 0.0004563058722294493, 1.727783238060185e-5, -0.0030530578694831466, 0.8605990248674548, -0.0015265015766455392, 0.007397659

In [124]:
function φSYS2(Ψ,ΣF,rf,γ)

    ## Notes

        ## TODO: Generalize: All RV's have zero mean.
        ## Z is random supply not demand

    # 00. Extract Variables and Define Random Variables

        a11,a12,a13,a14,a21,a22,a23,a24 = Ψ

        # F = [Π1, Π2, Z1, Z2, ε1i, ε2i]

    # 0. Define Private Information

        S_PRIVATE1 = [1,0,0,0,1,0]     # Signal Y1: F.S_PRIVATE1
        S_PRIVATE2 = [0,1,0,0,0,1]     # Signal Y1: F.S_PRIVATE1

    # 1. Conjecture Price Function (Public Information)

        ## P1 = a11*Π1 + a12*Π2 + a13*Z1 + a14*Z2
        ## P2 = a21*Π1 + a22*Π2 + a23*Z1 + a24*Z2

        S_PUBLIC1 = [a11, a12, a13, a14, 0, 0]          # Signal P1: F.S_PUBLIC
        S_PUBLIC2 = [a21, a22, a23, a24, 0, 0]          # Signal P1: F.S_PUBLIC

    # 2. Learning

        ## E[Π1|S] = b11*Y1 + b12*Y2 + b13*P1 + b14*P2
        ## E[Π2|S] = b21*Y1 + b22*Y2 + b23*P1 + b24*P2

        S = hcat(S_PRIVATE1, S_PRIVATE2, S_PUBLIC1, S_PUBLIC2)

        ΣYY = transpose(S) * ΣF * S                # 4x4
        ΣFY = ΣF * S                               # 6×4
        B = ΣFY * inv(ΣYY)                         # 6×4
        Σ = ΣF - ΣFY * inv(ΣYY) * transpose(ΣFY)   # 6×6

        B1 = B[1:2,1:2]
        B2 = B[1:2,3:4]
        ΣΠ = Σ[1:2,1:2]

    # 3. Individual Demand (CARA)

        ## Xi = {ΣΠ⁻¹ . [E[Π|S] - P*(1+rf)]} / γ
        ##    = ΣΠ⁻¹ . {b1*Y + [b2 - I*(1+rf)]*P} / γ
        ##    = C1*Y1 + C2*P

        C1 = inv(ΣΠ) * B1 / γ
        C2 = inv(ΣΠ) * (B2 - (1+rf)*I(2)) / γ

    # 4. Demand Aggregation

        ## Due to i.i.d. εi, μi aggregates to Π, therefore:
        ## Z = C1*Π + C2*P

    # 5. Market Clearing

        ## P = (-inv(C2)*C1)*Π + inv(C2)*Z
        ##   = A1*Π + A2*Z

        A1 = [a11 a12;a21 a22]
        A2 = [a13 a14;a23 a24]

        EQ_MC = [A1 - (-inv(C2)*C1), A2 - inv(C2)]

    return vec(vcat(EQ_MC[1],EQ_MC[2]))

end

φSYS2 (generic function with 1 method)

In [125]:
φSYS2(rand(8),diagm([1,1,1,1,1,1]),0,1)


8-element Vector{Float64}:
 -0.177868383955613
  0.0774101406562438
  1.0172684953294246
  0.40283524267865356
  1.0384150468506343
  0.3571378622223974
  0.3023951453500945
  0.8490299728594911

In [135]:
x0 = [0.999,0,-0.999,0,0,0.999,0,-0.999]
sol = nlsolve(x -> φSYS2(x, diagm([1,1,1,1,1,1]),0,1),x0)

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.999, 0.0, -0.999, 0.0, 0.0, 0.999, 0.0, -0.999]
 * Zero: [0.6666666666666661, 0.0, -0.6666666666666664, 0.0, 0.0, 0.6666666666666661, 0.0, -0.6666666666666664]
 * Inf-norm of residuals: 0.000000
 * Iterations: 5
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 6
 * Jacobian Calls (df/dx): 6